In [2]:
# File System
import os, sys, pickle as pkl
from os import path as pth
import itertools as it
from glob import glob

# Plotting
from matplotlib import pyplot as plt
from matplotlib import ticker as mticker
from corner import corner

# Math
import numpy as np
from numpy import linalg as la
import pandas as pd
pd.options.mode.chained_assignment = None

from scipy.stats import norm, uniform, multivariate_normal
import scipy.optimize as opt
from functools import reduce, partial

#Data Structs
from collections import defaultdict

# Display
from IPython.display import clear_output
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
import logging as lg
lg.basicConfig(level=lg.INFO)

sys.path.append(pth.join(".."))
from Model import TwoSignal, Signal
from mcmc import MHMCMC

data_path = "data"

In [4]:
data = dict()

with open(glob(pth.join(data_path, "*train*"))[0],'rb') as tr\
    ,open(glob(pth.join(data_path, "*valid*"))[0],'rb') as vd:
        data['train'] = pkl.load(tr)
        data['valid'] = pkl.load(vd)

{'bp_coef': array([[ 5.07578286e+04, -5.17054450e+03, -1.20813118e+03, ...,
         -2.27849615e-01, -1.50263621e+00, -1.18399576e+00],
        [ 1.47809559e+04, -6.93234003e+00, -7.36025117e+02, ...,
         -1.30689399e-01, -8.06343107e-02,  2.01984027e-01],
        [ 1.23582628e+04,  1.15931915e+02, -5.88475743e+02, ...,
         -2.35552324e-01,  1.02886697e-01, -8.56639073e-02],
        ...,
        [ 2.33231583e+04, -5.19157281e+02, -9.87775213e+02, ...,
          2.31345808e-01, -1.56367395e-01, -3.98727844e-02],
        [ 1.18620840e+05, -3.32809943e+03, -5.32420266e+03, ...,
          3.91741591e+01, -1.00676373e+01,  5.60543737e-01],
        [ 4.56386940e+04, -9.18422099e+03,  5.01602007e+02, ...,
         -1.18198395e+01, -7.08529864e-01,  6.33268687e+00]]),
 'rp_coef': array([[ 5.53155202e+04, -7.22708471e+03, -2.53207539e+02, ...,
          1.15257240e+00,  8.25811293e-01,  5.98844318e-01],
        [ 2.00417429e+04, -1.54443900e+03,  4.15355952e+01, ...,
          2.0268